# Module 4: NetApp Trident Use Scenarios

## Exercise 1: Managing storage using NetApp Trident

In this exercise, you explore and manage point-in-time Snapshot copies of a persistent volume (PV).

You also expand a PV. 

You also import a volume (that NetApp NetApp Trident does not control) as a PV that NetApp Trident manages. 

Finally, you will implement a multiple zone storage network.

**Objectives**

This exercise focuses on enabling you to do the following:
  
  - Manage Snapshot copies
  
  - Restore Snapshot data
  
  - Import an external Snapshot copy
  
  - Expand volumes
  
  - Import volumes
  
  - Implement multiple zones storage network

**Exercise Equipment**

In this exercise, you use the following systems.

| System                  | Host Name   | IP Addresses   | User Name (case sensitive) | Password  |
|-------------------------|-------------|----------------|----------------------------|-----------|
| Linux Mint 20           | jumphost    | 192.168.0.5    | user                       | Netapp1!  |
| Kubernetes Control Plane| kubmas1-1   | 192.168.0.61   | root                       | Netapp1!  |


**Prerequisites**
Before starting this exercise, you should take the following actions:
  
  - Set up your Integrated Development Environment (IDE)
  
  - Download the courseware GIT repository
  
  - Configure your IDE to have access to your Kubernetes clusters
  
  - Create svm0 and svm1
  
  - Configure svm0 to use the NFS v3 protocol
  
  - Configure svm1 to use the iSCSI and NVMe protocols
  
  - Install Trident in your source Kubernetes cluster
  
  - Ensure iSCSI, NVMe and NFS are properly configured on your worker nodes in the source Kubernetes cluster
  
  - Create the following TBE: c1-svm0-nfs-tbe, c1-svm1-iscsi-tbe, c1-svm0-nfs-custom-tbe
  
  - Create the following StorageClass: c1-svm0-nfs-sc, c1-svm1-iscsi-sc, c1-svm0-nfs-custom-sc
  
  - Create the following PVCs: c1-svm0-nfs-pvc-1, c1-svm1-iscsi-pvc-1, c1-svm0-nfs-custom-pvc-1
  
  - Provision the following PODs: nfs-pod, san-pod


---
---


#### Task 1: Manage Snapshot copies

In this task, you configure the required custom resource definitions (CRDs) to use point-in-time
Snapshot copies of volumes in Kubernetes. Then you create the snapshot controller.


---

In your integrated development environment (IDE), make sure that: 

you are in the Exercise 4 folder of the course contents.



In [1]:
pwd

/home/user/repos/STRSW-ILT-UATWK/Exercise 4


In your integrated development environment (IDE), make sure that you are using the **default** Namespace: 

![Use Default Namespace](../images/use_ns.jpg)





---

Create the volumeshapshotclass CRD:

[exercise4Task1-1.yaml](./exercise4Task1-1.yaml)

In [2]:
kubectl create -f exercise4Task1-1.yaml


customresourcedefinition.apiextensions.k8s.io/volumesnapshotclasses.snapshot.storage.k8s.io created


---

Create the volumesnapshotcontents CRD:

[exercise4Task1-2.yaml](./exercise4Task1-2.yaml)


In [3]:

kubectl create -f exercise4Task1-2.yaml


customresourcedefinition.apiextensions.k8s.io/volumesnapshotcontents.snapshot.storage.k8s.io created


---

Create the volumesnapshot CRD:

[exercise4Task1-3.yaml](./exercise4Task1-3.yaml)


In [4]:
kubectl create -f exercise4Task1-3.yaml


customresourcedefinition.apiextensions.k8s.io/volumesnapshots.snapshot.storage.k8s.io created


---

Create the service account, roles, and role bindings for the snapshot-controller in the kube-system namespace:

[exercise4Task1-4.yaml](./exercise4Task1-4.yaml)


In [5]:

kubectl create -f exercise4Task1-4.yaml


serviceaccount/snapshot-controller created
clusterrole.rbac.authorization.k8s.io/snapshot-controller-runner created
clusterrolebinding.rbac.authorization.k8s.io/snapshot-controller-role created
role.rbac.authorization.k8s.io/snapshot-controller-leaderelection created
rolebinding.rbac.authorization.k8s.io/snapshot-controller-leaderelection created


---

Create the snapshot-controller stateful set in the kube-system namespace:

[exercise4Task1-5.yaml](./exercise4Task1-5.yaml)


In [6]:
kubectl create -f exercise4Task1-5.yaml


deployment.apps/snapshot-controller created


---

Create a volumesnapshotclass custom resource (CR) that points to the NetApp Trident Container Storage Interface (CSI) driver:

[exercise4Task1-6.yaml](./exercise4Task1-6.yaml)


In [7]:
kubectl create -f exercise4Task1-6.yaml


volumesnapshotclass.snapshot.storage.k8s.io/csi-snap-class created


NOTE: 

The `deletionPolicy` can be **Delete** or **Retain**. 

When set to **Retain**, the underlying physical snapshot on the storage cluster is retained even when the `VolumeSnapshot` object is deleted.


---

Edit the snapshot definition in the [exercise4Task1-7.yaml](./exercise4Task1-7.yaml) file, 

and then save the file as **exercise4Task1-7mod.yaml**.


Under spec:
 - volumeSnapshotClassName: [name of the snapshotclass that you created]
 - source persistentVolumeClaimName: [name of the NFS-backed persistent volume claim (PVC) that you created in the previous exercise ]


<details> <summary>Solution </summary>

[exercise4Task1-7.yaml](./Solutions/exercise4Task1-7.yaml)

  
```yaml
apiVersion: snapshot.storage.k8s.io/v1
kind: VolumeSnapshot
metadata:
  name: c1-svm0-nfs-pvc-1-snap-1
  namespace: default
spec:
  volumeSnapshotClassName: csi-snap-class
  source:
    persistentVolumeClaimName: c1-svm0-nfs-pvc-1
    

---

Create a snapshot of the NFS-backed PVC that you created in the previous exercise:


In [8]:
kubectl create -f exercise4Task1-7mod.yaml


volumesnapshot.snapshot.storage.k8s.io/c1-svm0-nfs-pvc-1-snap-1 created


---

Verify that you created the snapshot:


In [10]:
kubectl -n default get volumesnapshots


NAME                       READYTOUSE   SOURCEPVC           SOURCESNAPSHOTCONTENT   RESTORESIZE   SNAPSHOTCLASS    SNAPSHOTCONTENT                                    CREATIONTIME   AGE
c1-svm0-nfs-pvc-1-snap-1   true         c1-svm0-nfs-pvc-1                           348Ki         csi-snap-class   snapcontent-7a949c76-5f27-4264-a766-60faeb9bf8d2   24m            24m


Sample output:
```
NAME       READYTOUSE   SOURCEPVC … RESTORESIZE  SNAPSHOTCLASS   SNAPSHOTCONTENT
c1-svm0-n… true         c1-svm0-n … 272Ki        csi-snap-class  snapcontent-b6d…


---

Describe the snapshot, and notice the **Ready to Use** parameter:


In [11]:
kubectl describe volumesnapshots c1-svm0-nfs-pvc-1-snap-1


Name:         c1-svm0-nfs-pvc-1-snap-1
Namespace:    default
Labels:       <none>
Annotations:  <none>
API Version:  snapshot.storage.k8s.io/v1
Kind:         VolumeSnapshot
Metadata:
  Creation Timestamp:  2025-03-29T02:20:06Z
  Finalizers:
    snapshot.storage.kubernetes.io/volumesnapshot-as-source-protection
    snapshot.storage.kubernetes.io/volumesnapshot-bound-protection
  Generation:        1
  Resource Version:  1024753
  UID:               7a949c76-5f27-4264-a766-60faeb9bf8d2
Spec:
  Source:
    Persistent Volume Claim Name:  c1-svm0-nfs-pvc-1
  Volume Snapshot Class Name:      csi-snap-class
Status:
  Bound Volume Snapshot Content Name:  snapcontent-7a949c76-5f27-4264-a766-60faeb9bf8d2
  Creation Time:                       2025-03-29T02:19:56Z
  Ready To Use:                        true
  Restore Size:                        348Ki
Events:
  Type    Reason            Age   From                 Message
  ----    ------            ----  ----                 -------
  Normal  Cre

In [15]:
kubectl describe volumesnapshots c1-svm0-nfs-pvc-1-snap-1 |grep -C5 -i 'ready to use'


    Persistent Volume Claim Name:  c1-svm0-nfs-pvc-1
  Volume Snapshot Class Name:      csi-snap-class
Status:
  Bound Volume Snapshot Content Name:  snapcontent-7a949c76-5f27-4264-a766-60faeb9bf8d2
  Creation Time:                       2025-03-29T02:19:56Z
  Ready To Use:                        true
  Restore Size:                        348Ki
Events:
  Type    Reason            Age   From                 Message
  ----    ------            ----  ----                 -------
  Normal  CreatingSnapshot  32m   snapshot-controller  Waiting for a snapshot default/c1-svm0-nfs-pvc-1-snap-1 to be created by the CSI driver.
  Normal  SnapshotCreated   32m   snapshot-controller  Snapshot default/c1-svm0-nfs-pvc-1-snap-1 was successfully created by the CSI driver.
  Normal  SnapshotReady     32m   snapshot-controller  Snapshot default/c1-svm0-nfs-pvc-1-snap-1 is ready to use.


Sample output:
```
Name: c1-svm0-nfs-pvc-1-snap-1
Namespace: default
Labels: <none>
Annotations: <none>
API Version: snapshot.storage.k8s.io/v1
Kind: VolumeSnapshot
Metadata:
Creation Timestamp: 2024-11-19T20:43:52Z
Finalizers:
snapshot.storage.kubernetes.io/volumesnapshot-as-source-protection
snapshot.storage.kubernetes.io/volumesnapshot-bound-protection
Generation: 1
Resource Version: 205488
UID: cff5a387-6d10-4616-a9c2-de6b9a9d268d
Spec:
Source:
Persistent Volume Claim Name: c1-svm0-nfs-pvc-1
Volume Snapshot Class Name: csi-snap-class
Status:
Bound Volume Snapshot Content Name: snapcontent-cff5a387-6d10-4616-a9c2-de6b9a9d268d
Creation Time: 2024-11-19T20:44:55Z
Ready To Use: true
Restore Size: 272Ki
Events:
Type Reason Age From Message
---- ------ ---- ---- -------
Warning GetSnapshotClassFailed 3m45s snapshot-controller Failed to get snapshot class with error
volumesnapshotclass.snapshot.storage.k8s.io "csi-snap-class" not found
Normal CreatingSnapshot 2m44s snapshot-controller Waiting for a snapshot default/c1-svm0-
nfs-pvc-1-snap-1 to be created by the CSI driver.
Normal SnapshotCreated 2m43s snapshot-controller Snapshot default/c1-svm0-nfs-pvc-1-
snap-1 was successfully created by the CSI driver.
Normal SnapshotReady 2m43s snapshot-controller Snapshot default/c1-svm0-nfs-pvc-1-
snap-1 is ready to use.
```

---

Use the following defintions to complete a PVC in the [exercise4Task1-8.yaml](./exercise4Task1-8.yaml) file, 

and save it as **exercise4Task1-8mod.yaml**

then execute the YAML:
  
  - Metadata name: c1-svm0-nfs-pvcsnap-1
  - Spec:
    - accessModes: ReadWriteOnce
    - resource requests storage: 1Gi
    - storageClassName: c1-svm0-nfs-sc
    - dataSource:
      - name: c1-svm0-nfs-pvc-1-snap-1
      - kind: VolumeSnapshot
      - apiGroup: snapshot.storage.k8s.io

**OR**

in dot notation:

```
metadata.name: c1-svm0-nfs-pvcsnap-1
metadata.namespace: default
spec.accessModes[0]: ReadWriteOnce
spec.resources.requests.storage: 1Gi
spec.storageClassName: c1-svm0-nfs-sc
spec.dataSource.name: c1-svm0-nfs-pvc-1-snap-1
spec.dataSource.kind: VolumeSnapshot
spec.dataSource.apiGroup: snapshot.storage.k8s.io
```

<details> <summary>Solution </summary>

[exercise4Task1-8.yaml](./Solutions/exercise4Task1-8.yaml)

```yaml
kind: PersistentVolumeClaim
apiVersion: v1
metadata:
  name: c1-svm0-nfs-pvcsnap-1
  namespace: default
spec:
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 1Gi
  storageClassName: c1-svm0-nfs-sc
  dataSource:
    name: c1-svm0-nfs-pvc-1-snap-1
    kind: VolumeSnapshot
    apiGroup: snapshot.storage.k8s.io

---

In [18]:
kubectl create -f exercise4Task1-8mod.yaml

persistentvolumeclaim/c1-svm0-nfs-pvcsnap-1 created


Verify that you created the PVC:


In [19]:
kubectl -n default get pvc


NAME                       STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS            VOLUMEATTRIBUTESCLASS   AGE
c1-svm0-nfs-custom-pvc-1   Bound    pvc-0a43a259-011b-4044-9ffa-230e7310f378   1Gi        RWO            c1-svm0-nfs-custom-sc   <unset>                 12h
c1-svm0-nfs-eco-pvc-1      Bound    pvc-bcdaaa4b-b291-4454-bc51-d46d070a9dda   1Gi        RWO            c1-svm0-nfs-eco-sc      <unset>                 9h
c1-svm0-nfs-eco-pvc-2      Bound    pvc-17c32847-b2f4-4e88-a88c-605a2b5d9f0a   1Gi        RWO            c1-svm0-nfs-eco-sc      <unset>                 9h
c1-svm0-nfs-pvc-1          Bound    pvc-c06ed368-dd15-45c1-bb53-7ff66c8c40f8   1Gi        RWO            c1-svm0-nfs-sc          <unset>                 28h
c1-svm0-nfs-pvcsnap-1      Bound    pvc-de4de6ba-e2c2-4a57-8333-9ba8f2580ece   1Gi        RWO            c1-svm0-nfs-sc          <unset>                 8s
c1-svm1-iscsi-pvc-1        Bound    pvc-cb4cfb2f-914d-4848-9c

Sample output:
```
NAME                  STATUS    VOLUME                 CAPACITY ACCESS MODES STORAGECLASS VOLUMEATTRIBUT…
…
c1-svm0-nfs-pvc-1     Bound     pvc-5ea27d4            1Gi      RWO          c1-svm0-nfs-sc <unset>
c1-svm0-nfs-pvcsnap-1 Bound     pvc-af67c22            1Gi      RWO          c1-svm0-nfs-sc <unset>



---

Describe the PVC that you created:


In [20]:
kubectl -n default describe pvc c1-svm0-nfs-pvcsnap-1


Name:          c1-svm0-nfs-pvcsnap-1
Namespace:     default
StorageClass:  c1-svm0-nfs-sc
Status:        Bound
Volume:        pvc-de4de6ba-e2c2-4a57-8333-9ba8f2580ece
Labels:        <none>
Annotations:   pv.kubernetes.io/bind-completed: yes
               pv.kubernetes.io/bound-by-controller: yes
               volume.beta.kubernetes.io/storage-provisioner: csi.trident.netapp.io
               volume.kubernetes.io/storage-provisioner: csi.trident.netapp.io
Finalizers:    [kubernetes.io/pvc-protection]
Capacity:      1Gi
Access Modes:  RWO
VolumeMode:    Filesystem
DataSource:
  APIGroup:  snapshot.storage.k8s.io
  Kind:      VolumeSnapshot
  Name:      c1-svm0-nfs-pvc-1-snap-1
Used By:     <none>
Events:
  Type    Reason                 Age   From                                                                                            Message
  ----    ------                 ----  ----                                                                                            -------


---

NOTE:

For this operation of creating a PVC based upon an ONTAP Snapshot copy, the ONTAP cluster requires a NetApp FlexClone license. 

The PVC status would be pending until you install the FlexClone license.


---

Create a pod manifest similar to [exercise3Task4-1.yaml](../Exercise%203/exercise3Task4-1.yaml), 

name the new definition **exercise4Task1-9mod.yaml**, 

and enter the following attributes:
  - Metadata name: nfs-snap-pod
  - Metadata label: app: nfs-snap-web
  - Spec volumes persistentVolumeClaim claimName: nfs-snap

Execute the new pod definition.

<details> <summary>Solution </summary> 

[exercise4Task1-9.yaml](./Solutions/exercise4Task1-9.yaml)
 ```yaml

---

In [21]:
kubectl create -f exercise4Task1-9mod.yaml

pod/nfs-snap-pod created


CHALLENGE STEP: 

Create a NodePort and view your custom webpage. 

If desired, you can change the custom webpage to differentiate it from the previous exercise’s NFS-backed pod.

See [exercise4Task1-10.yaml](./Solutions/exercise4Task1-10.yaml) for assistance.



---

CHALLENGE STEP: 

Edit [exercise4Task1-7.yaml](exercise4Task1-7.yaml) for the SAN-backed PVC and pod. 

You can find solutions in the 


[exercise4Task1-11.yaml](./Solutions/exercise4Task1-11.yaml)

[exercise4Task1-12.yaml](./Solutions/exercise4Task1-12.yaml)

[exercise4Task1-13.yaml](./Solutions/exercise4Task1-13.yaml)

[exercise4Task1-14.yaml](./Solutions/exercise4Task1-14.yaml)


---
---


#### Task 2: Restore Snapshot data

In this task, you will restore snapshot data of a NFS-attached persistent volume. 

Take note that in the [exercise3Task3.yaml](../Exercise%203/exercise3Task3.yaml) file, there is an annotation that reveals the snapshot directory, which is not visible by default.


---

Identify the name of PV volume used for the PVC of the nfs-pod.


---

Verify that the nfs-pod still exists:


In [23]:
kubectl -n default get pod nfs-pod



NAME      READY   STATUS    RESTARTS   AGE
nfs-pod   1/1     Running   0          24h


NOTE: 

If it doesn’t exists, please recreate it by using [exercise3Task4-1.yaml](../Exercise%203/exercise3Task4-1mod.yaml) file.


---

Verify that the nfs-web service still exists:


In [24]:
kubectl -n default get services nfs-web


NAME      TYPE       CLUSTER-IP     EXTERNAL-IP   PORT(S)        AGE
nfs-web   NodePort   10.104.74.17   <none>        80:32738/TCP   28h


NOTE: 

If it doesn’t exists, please recreate it by using [exercise3Task4-2.yaml](../Exercise%203/exercise3Task4-2.yaml) file.


---

Open a browser to one of the Kubernetes cluster node IP addresses and the NodePort
referenced in the previous step. 

`For example: http://192.168.0.62:31319`. 

You should see you index.html you created previously in the browser.


In [25]:
nodeport=$(kubectl -n default describe service nfs-web|grep -oP 'NodePort:\s+<unset>\s+\K\d+')

echo "http://kubwor1-1:$nodeport"

http://kubwor1-1:32738


---

Connect to the pod:


`kubectl -n default exec -it nfs-pod -- /bin/sh
`

---

Change the directory to the Trident persistent volume:

`# cd /usr/share/nginx/html`

---

Update file in the current directory:

`# echo '<html><body>Creating the file</body></html>' > index.html
`

---

In [ ]:
kubectl -n default exec -it nfs-pod -- /bin/sh -c "echo '<html><body>Creating the file</body></html>' > /usr/share/nginx/html/index.html"

Display the webpage and verify the webpage contains your updated message using the
nodeport service.

You may need to refresh the page to remove any cached pages

In [28]:
nodeport=$(kubectl -n default describe service nfs-web|grep -oP 'NodePort:\s+<unset>\s+\K\d+')

echo "http://kubwor1-1:$nodeport"

http://kubwor1-1:32738


---

Create another snapshot with containing the newly created index.html file:


In [29]:
kubectl create -f exercise4Task2.yaml

volumesnapshot.snapshot.storage.k8s.io/c1-svm0-nfs-pvc-1-snap-2 created


Update file in the current directory:

`# echo '<html><body>Updating the file</body></html>' > index.html
`

In [30]:
kubectl -n default exec -it nfs-pod -- /bin/sh -c "echo '<html><body>Updating the file</body></html>' > /usr/share/nginx/html/index.html"

---

Display the webpage and verify the webpage contains your updated message using the
nodeport service.


In [31]:
nodeport=$(kubectl -n default describe service nfs-web|grep -oP 'NodePort:\s+<unset>\s+\K\d+')

echo "http://kubwor1-1:$nodeport"

http://kubwor1-1:32738


---

List the hidden snapshot directory in the /usr/share/nginx/html path:

`# ls .snapshot
`

Sample output:
```
snapshot-49593425-ece8-4874-9e1c-733d7218ae2d
```

In [38]:
kubectl -n default exec -it nfs-pod -- /bin/sh -c "ls -l /usr/share/nginx/html/.snapshot"

total 8
drwxrwxrwx    2 nobody   nobody        4096 Mar 27 23:07 snapshot-55376c02-0a40-4223-9686-c1a50ed6d2cb
drwxrwxrwx    2 nobody   nobody        4096 Mar 27 23:07 snapshot-7a949c76-5f27-4264-a766-60faeb9bf8d2


---

Change directory into the Snapshot copy created in the previous task (your snapshot name will
vary):

`# cd .snapshot/snapshot-49593425-ece8-4874-9e1c-733d7218ae2d
`

---

Verify you have access to the previous version of the index.html:

`# cat index.html
`

NOTE:

The original message should be in the html page: Creating the file.


In [71]:
snap1=$(kubectl -n default exec -it nfs-pod -- /bin/sh -c "ls /usr/share/nginx/html/.snapshot"|awk 'NR==1{print $1}'); file1="/usr/share/nginx/html/$snap1/index.html"

In [81]:
file1="/usr/share/nginx/html/${snap1}/index.html"; echo "${file1}"; #kubectl -n default exec -it nfs-pod -- /bin/sh -c "cat $file1"

/index.htmlnginx/html/snapshot-55376c02-0a40-4223-9686-c1a50ed6d2cb


In [78]:
echo "$snap1" ; file1="/usr/share/nginx/html/${snap1}/index.html"; echo "$file1"

snapshot-55376c02-0a40-4223-9686-c1a50ed6d2cb
/index.htmlnginx/html/snapshot-55376c02-0a40-4223-9686-c1a50ed6d2cb


In [43]:
kubectl -n default exec -it nfs-pod -- /bin/sh -c 'echo "/usr/share/nginx/html/.snapshot/$snap1/index.html"'

/usr/share/nginx/html/.snapshot//index.html


---

Copy the previous version to the active file system:
# cp index.html ../../index.html
2-16 Verify the webpage as updated to the first custom message.
2-17 CHALLENGE STEP: Change the message again in the index.html file. Restore the entire
volume using Snapshot restore.
Task 3: Import an external Snapshot copy
In this task, you will import a snapshot copy taken outside of Trident and provide it as volume snapshot
in the Kubernetes cluster.
Step Action
3-1 Using System Manager or the ONTAP CLI, create a snapshot named snap.1 for
Cluster1_default_c1_svm0_nfs-custom_pvc_1_c1_svm0_nfs_custom_tbe_[unique id] volume.
3-2 In the exercise4Task3-1.yaml file, replace the change_me in the snapshotHandle field
with the name of the Persistent Volume (PV) associated with volume used in the previous step.NetApp Learning Services 